In [2]:
import pandas as pd
import boto3
from StringIO import StringIO
from sklearn.feature_extraction.text import CountVectorizer
from helper_functions import split_and_add_to_set, clean_value
from text_processing import run_sklearn_nmf, custom_tokenizer
pd.options.mode.chained_assignment = None

### Init s3 client

In [3]:
s3 = boto3.client('s3') 

### Load Tickets

In [4]:
tickets_csv_string = s3.get_object(Bucket='braydencleary-data', Key='feastly/cleaned/tickets.csv')['Body'].read().decode('utf-8')
tickets = pd.read_csv(StringIO(tickets_csv_string), header=0, delimiter='|')
tickets['meal_created_date'] = pd.to_datetime(tickets['meal_created_date'])
tickets['meal_date'] = pd.to_datetime(tickets['meal_date'])

### Load Meals

In [5]:
meals_csv_string = s3.get_object(Bucket='braydencleary-data', Key='feastly/cleaned/meals.csv')['Body'].read().decode('utf-8')
meals = pd.read_csv(StringIO(meals_csv_string), header=0)

### Load Cooks

In [6]:
cooks_csv_string = s3.get_object(Bucket='braydencleary-data', Key='feastly/cleaned/cooks.csv')['Body'].read().decode('utf-8')
cooks = pd.read_csv(StringIO(cooks_csv_string), header=0)

### Load Menus

In [7]:
menus_csv_string = s3.get_object(Bucket='braydencleary-data', Key='feastly/cleaned/menus.csv')['Body'].read().decode('utf-8')
menus = pd.read_csv(StringIO(menus_csv_string), header=0)

### Load Venues

In [8]:
venues_csv_string = s3.get_object(Bucket='braydencleary-data', Key='feastly/cleaned/venues.csv')['Body'].read().decode('utf-8')
venues = pd.read_csv(StringIO(venues_csv_string), header=0)

### Load Menu Course Counts

In [9]:
menu_course_counts_csv_string = s3.get_object(Bucket='braydencleary-data', Key='feastly/cleaned/menu_course_counts.csv')['Body'].read().decode('utf-8')
menu_course_counts = pd.read_csv(StringIO(menu_course_counts_csv_string), header=0, delimiter='|')

### Load Meal Addon Counts

In [10]:
meal_addon_counts_csv_string = s3.get_object(Bucket='braydencleary-data', Key='feastly/cleaned/meal_addon_counts.csv')['Body'].read().decode('utf-8')
meal_addon_counts = pd.read_csv(StringIO(meal_addon_counts_csv_string), header=0, delimiter='|')

### Load meal_inferred_types

In [11]:
meal_inferred_types_csv_string = s3.get_object(Bucket='braydencleary-data', Key='feastly/cleaned/meal_inferred_types.csv')['Body'].read().decode('utf-8')
meal_inferred_types = pd.read_csv(StringIO(meal_inferred_types_csv_string), header=0, delimiter='|')

### Load meal_is_interactive 

In [12]:
meal_is_interactive_csv_string = s3.get_object(Bucket='braydencleary-data', Key='feastly/cleaned/meal_is_interactive.csv')['Body'].read().decode('utf-8')
meal_is_interactive = pd.read_csv(StringIO(meal_is_interactive_csv_string), header=0, delimiter='|')

### Load meal_cuisine_types

In [13]:
meal_cuisine_types_csv_string = s3.get_object(Bucket='braydencleary-data', Key='feastly/cleaned/meal_cuisine_types.csv')['Body'].read().decode('utf-8')
meal_cuisine_types = pd.read_csv(StringIO(meal_cuisine_types_csv_string), header=0, delimiter='|')

### Filter meals to only include those present in tickets df

In [14]:
meals = meals[meals.id.isin(tickets['meal_id'].unique())]

### Set initial feature matrix (to be expanded on as notebook progresses)

##### Including meal_id, percentage_of_seats_sold, sold (target), days_to_sell, and meal_date in feature matrix now but will remove later

In [15]:
X = tickets[['meal_id', 'percentage_of_seats_sold', 'sold', 'meal_date', 'days_to_sell', 'number_of_seats', 'ticket_price']]

### Calculate listed_days feature and add to feature matrix

In [16]:
tickets['listed_days'] = (tickets['meal_date'] - tickets['meal_created_date'])
tickets['listed_days'] = tickets['listed_days'].apply(lambda listed_days: listed_days.days)
X['meal_listed_days'] = tickets['listed_days']

### Calculate meal day of week dummies and add to feature matrix

In [17]:
tickets['meal_day_of_week'] = tickets['meal_date'].apply(lambda x: x.weekday())
X['meal_is_on_monday'] = tickets['meal_day_of_week'].apply(lambda x: 1 if x == 0 else 0)
X['meal_is_on_tuesday'] = tickets['meal_day_of_week'].apply(lambda x: 1 if x == 1 else 0)
X['meal_is_on_wednesday'] = tickets['meal_day_of_week'].apply(lambda x: 1 if x == 2 else 0)
X['meal_is_on_thursday'] = tickets['meal_day_of_week'].apply(lambda x: 1 if x == 3 else 0)
X['meal_is_on_friday'] = tickets['meal_day_of_week'].apply(lambda x: 1 if x == 4 else 0)
X['meal_is_on_saturday'] = tickets['meal_day_of_week'].apply(lambda x: 1 if x == 5 else 0)
X['meal_is_on_sunday'] = tickets['meal_day_of_week'].apply(lambda x: 1 if x == 6 else 0)
X['meal_is_on_weekday'] = tickets['meal_day_of_week'].apply(lambda x: 1 if x < 5 else 0)

### Calculate cook days on platform and add to feature matrix

In [18]:
def compute_cook_days_on_platform(row):
    if (row['meal_date'] - row['cook_joined_date']).days < 0:
        return None
    else:
        return (row['meal_date'] - row['cook_joined_date']).days

In [19]:
cooks['joined_date'] = pd.to_datetime(cooks['joined_date'])
X['cook_joined_date'] = cooks['joined_date']
X['cook_days_on_platform'] = X.apply(compute_cook_days_on_platform, axis=1)
X['couldnt_compute_cook_days_on_platform'] = X['cook_days_on_platform'].apply(lambda x: 0 if x > 0 else 1)

### Add dummies for venue style to feature matrix

In [20]:
unqiue_venue_style = set()

venues['venue_style'].apply(split_and_add_to_set, args=(unqiue_venue_style, ',', ))

for category in unqiue_venue_style:
    if len(category) > 0:
        column_name = 'is_venue_style_' + clean_value(category)
        venues[column_name] = venues['venue_style'].apply(lambda x: 1 if category in x.split(',') else 0)

In [21]:
X = pd.merge(X, meals[['id','venue_id']], left_on='meal_id', right_on='id', how='inner')
del X['id']
X = pd.merge(X, venues[['id', 'is_venue_style_pop-up-space', 'is_venue_style_apartment', 'is_venue_style_restaurant', 'is_venue_style_farm', 'is_venue_style_house', 'is_venue_style_brown-stone']], left_on='venue_id', right_on='id', how='inner')
del X['id']
del X['venue_id']


### Add menu course count columns to feature matrix

In [22]:
X = pd.merge(X, meals[['id','menu_id']], left_on='meal_id', right_on='id', how='inner')
del X['id']
X = pd.merge(X, menu_course_counts[['menu_id', 'count_of_first_courses', 'count_of_second_courses', 'count_of_third_courses', 'count_of_appetizers', 'count_of_desserts', 'count_of_small_plates', 'count_of_entrees', 'count_of_beverages']], on='menu_id', how='inner')
del X['menu_id']

### Add addon counts/info to feature matrix (this is a bit questionable becuase I only have meal_addon_counts data for about 15% of meals...definitely remove and see if performance improves without)

In [23]:
X = pd.merge(X, meal_addon_counts[['meal_id', 'count_of_addons', 'total_price_of_addons', 'max_price_of_addons', 'min_price_of_addons']], on='meal_id', how='left')
X['missing_count_of_addons'] = X['count_of_addons'].apply(lambda x: 0 if x > 0 else 1)
X['count_of_addons'].fillna(X['count_of_addons'].mean(), inplace=True)
X['missing_total_price_of_addons'] = X['total_price_of_addons'].apply(lambda x: 0 if x > 0 else 1)
X['total_price_of_addons'].fillna(X['total_price_of_addons'].mean(), inplace=True)
X['missing_max_price_of_addons'] = X['max_price_of_addons'].apply(lambda x: 0 if x > 0 else 1)
X['max_price_of_addons'].fillna(X['max_price_of_addons'].mean(), inplace=True)
X['missing_min_price_of_addons'] = X['min_price_of_addons'].apply(lambda x: 0 if x > 0 else 1)
X['min_price_of_addons'].fillna(X['min_price_of_addons'].mean(), inplace=True)

### Add cooking experience years to feature matrix

In [24]:
cooks['cooking_experience_years'].fillna('missing', inplace=True)

unique_cooking_experience_years_values = set()

cooks['cooking_experience_years'].apply(split_and_add_to_set, args=(unique_cooking_experience_years_values, ',', ))

for category in unique_cooking_experience_years_values:
    if len(category) > 0:
        column_name = 'is_cooking_experience_years_' + clean_value(category)
        cooks[column_name] = cooks['cooking_experience_years'].apply(lambda x: 1 if category in x.split(',') else 0)

In [25]:
X = pd.merge(X, meals[['id','cook_id']], left_on='meal_id', right_on='id', how='inner')
del X['id']
X = pd.merge(X, cooks[['id', 'is_cooking_experience_years_8+', 'is_cooking_experience_years_1-3', 'is_cooking_experience_years_4-7', 'is_cooking_experience_years_0', 'is_cooking_experience_years_missing']], left_on='cook_id', right_on='id', how='inner')
del X['id']
del X['cook_id']

### Add cook referrer dummies to feature matrix

In [26]:
cooks['referrer'].fillna('missing', inplace=True)

unique_referrer_values = set()

cooks['referrer'].apply(split_and_add_to_set, args=(unique_referrer_values, ',', ))

for category in unique_referrer_values:
    if len(category) > 0:
        column_name = 'is_cook_referrer_' + clean_value(category)
        cooks[column_name] = cooks['referrer'].apply(lambda x: 1 if category in x.split(',') else 0)

In [27]:
X = pd.merge(X, meals[['id','cook_id']], left_on='meal_id', right_on='id', how='inner')
del X['id']
X = pd.merge(X, cooks[['id', 'is_cook_referrer_google', 'is_cook_referrer_missing', 'is_cook_referrer_job', 'is_cook_referrer_other', 'is_cook_referrer_social', 'is_cook_referrer_news', 'is_cook_referrer_referral', 'is_cook_referrer_meal', 'is_cook_referrer_friend']], left_on='cook_id', right_on='id', how='inner')
del X['id']
del X['cook_id']

### Adding cooking experience (qualitiative) dummies to feature matrix

In [28]:
cooks['cooking_experience'].fillna('missing', inplace=True)

unique_cooking_experience_values = set()

cooks['cooking_experience'].apply(split_and_add_to_set, args=(unique_cooking_experience_values, ' or ', ))

for category in unique_cooking_experience_values:
    if len(category) > 0:
        column_name = 'is_cooking_experience_' + clean_value(category)
        cooks[column_name] = cooks['cooking_experience'].apply(lambda x: 1 if category in x.split(',') else 0)

In [29]:
X = pd.merge(X, meals[['id','cook_id']], left_on='meal_id', right_on='id', how='inner')
del X['id']
X = pd.merge(X, cooks[['id', 'is_cooking_experience_current-chef', 'is_cooking_experience_avid', 'is_cooking_experience_former-chef', 'is_cooking_experience_missing', 'is_cooking_experience_home_cook', 'is_cooking_experience_ownerf', 'is_cooking_experience_novice', 'is_cooking_experience_private', 'is_cooking_experience_entrepreneur', 'is_cooking_experience_caterer', 'is_cooking_experience_chef_de_partie', 'is_cooking_experience_personal_chef', 'is_cooking_experience_chef_de_cuisine', 'is_cooking_experience_chef-in-traning', 'is_cooking_experience_commis', 'is_cooking_experience_sous_chef']], left_on='cook_id', right_on='id', how='inner')
del X['id']
del X['cook_id']

### Add cooks cooking reason dummies to feature matrix

In [30]:
cooks['is_cooking_reason_meet'] = cooks['is_reason_meet']
cooks['is_cooking_reason_brand'] = cooks['is_reason_brand']
cooks['is_cooking_reason_money'] = cooks['is_reason_money']

In [31]:
X = pd.merge(X, meals[['id','cook_id']], left_on='meal_id', right_on='id', how='inner')
del X['id']
X = pd.merge(X, cooks[['id', 'is_cooking_reason_meet', 'is_cooking_reason_brand', 'is_cooking_reason_money']], left_on='cook_id', right_on='id', how='inner')
del X['id']
del X['cook_id']

### Add menu_style dummies to feature matrix

In [32]:
unique_menu_style_values = set()

menus['menu_style'].apply(split_and_add_to_set, args=(unique_menu_style_values, ','))

for category in unique_menu_style_values:
    if len(category) > 0:
        column_name = 'is_menu_style_' + clean_value(category)
        menus[column_name] = menus['menu_style'].apply(lambda x: 1 if category in x.split(',') else 0)

In [33]:
X = pd.merge(X, meals[['id','menu_id']], left_on='meal_id', right_on='id', how='inner')
del X['id']
X = pd.merge(X, menus[['id', 'is_menu_style_not-defined', 'is_menu_style_finedining-elegant', 'is_menu_style_semicasual-upscale', 'is_menu_style_casual-homestyle']], left_on='menu_id', right_on='id', how='inner')
del X['id']
del X['menu_id']

### Add is_interactive dummy to feature matrix

In [34]:
X = pd.merge(X, meal_is_interactive[['meal_id','is_interactive']], on='meal_id', how='inner')

### Add inferred_type meal type to feature matrix (breakfast, brunch, lunch, dinner)

In [35]:
X = pd.merge(X, meal_inferred_types[['meal_id', 'is_inferred_by_text_columns_breakfast', 'is_inferred_by_text_columns_brunch', 'is_inferred_by_text_columns_lunch', 'is_inferred_by_text_columns_dinner']], on='meal_id', how='inner')

### Filter meals from feature matrix that have no cuisine type (134 out of 4800)

In [36]:
meals['cuisine_type'].fillna('', inplace=True)
meals_with_cuisine_type = meals[meals['cuisine_type'].apply(lambda x: len(x) > 0)]['id']
X = X[X['meal_id'].isin(meals_with_cuisine_type)]

### Add dummies for meal_cuisine_types in feature_matrix

##### First level of cuisine type

In [37]:
meal_cuisine_types['ct1'].fillna('', inplace=True)

unique_meal_ct1_values = set()

meal_cuisine_types['ct1'].apply(split_and_add_to_set, args=(unique_meal_ct1_values, ','))

for category in unique_meal_ct1_values:
    if len(category) > 0:
        column_name = 'is_ct1_' + clean_value(category)
        meal_cuisine_types[column_name] = meal_cuisine_types['ct1'].apply(lambda x: 1 if category in x.split(',') else 0)

###### Second level of cuisine type

In [38]:
meal_cuisine_types['ct2'].fillna('', inplace=True)

unique_meal_ct2_values = set()

meal_cuisine_types['ct2'].apply(split_and_add_to_set, args=(unique_meal_ct2_values, ','))

for category in unique_meal_ct2_values:
    if len(category) > 0:
        column_name = 'is_ct2_' + clean_value(category)
        meal_cuisine_types[column_name] = meal_cuisine_types['ct2'].apply(lambda x: 1 if category in x.split(',') else 0)

###### Third level of cuisine type

In [39]:
meal_cuisine_types['ct3'].fillna('', inplace=True)

unique_meal_ct3_values = set()

meal_cuisine_types['ct3'].apply(split_and_add_to_set, args=(unique_meal_ct3_values, ','))

for category in unique_meal_ct3_values:
    if len(category) > 0:
        column_name = 'is_ct3_' + clean_value(category)
        meal_cuisine_types[column_name] = meal_cuisine_types['ct3'].apply(lambda x: 1 if category in x.split(',') else 0)

In [40]:
X = pd.merge(X, meal_cuisine_types[['meal_id', 'is_ct1_portuguese', 'is_ct1_irish', 'is_ct1_mexican', 'is_ct1_chinese', 'is_ct1_german', 'is_ct1_chamorro', 'is_ct1_central_american', 'is_ct1_mediterranean', 'is_ct1_japanese', 'is_ct1_singaporean', 'is_ct1_desserts_bakeries', 'is_ct1_ecuadorian', 'is_ct1_persian', 'is_ct1_asian', 'is_ct1_latin_american', 'is_ct1_spanish', 'is_ct1_ice_cream_gelato', 'is_ct1_barbecue', 'is_ct1_cafe', 'is_ct1_brunch', 'is_ct1_paleo', 'is_ct1_caribbean', 'is_ct1_argentinian', 'is_ct1_vietnamese', 'is_ct1_tapas_small_plates', 'is_ct1_american', 'is_ct1_nordic', 'is_ct1_south_american', 'is_ct1_gastropub_food', 'is_ct1_peruvian', 'is_ct1_indian', 'is_ct1_guatemalan', 'is_ct1_brazilian', 'is_ct1_korean', 'is_ct1_health_food', 'is_ct1_european', 'is_ct1_indonesian', 'is_ct1_lao', 'is_ct1_hawaiian', 'is_ct1_jewish', 'is_ct1_african', 'is_ct1_middle_eastern', 'is_ct1_french', 'is_ct1_asian_noodle_soup', 'is_ct1_vegan', 'is_ct1_russian', 'is_ct1_thai', 'is_ct1_australian', 'is_ct1_other', 'is_ct1_balkan', 'is_ct1_cuban', 'is_ct1_filipino', 'is_ct1_east_european', 'is_ct1_seafood', 'is_ct1_turkish', 'is_ct1_malaysian', 'is_ct1_british', 'is_ct1_salvadorian', 'is_ct1_north_african', 'is_ct1_greek', 'is_ct1_burmese', 'is_ct1_hispanic', 'is_ct1_pizza', 'is_ct1_cajun_creole', 'is_ct1_north_american', 'is_ct1_californian', 'is_ct1_vegetarian', 'is_ct1_soul_food', 'is_ct1_italian', 'is_ct2_portuguese', 'is_ct2_irish', 'is_ct2_cajun_creole', 'is_ct2_chinese', 'is_ct2_german', 'is_ct2_hispanic', 'is_ct2_central_american', 'is_ct2_mediterranean', 'is_ct2_japanese', 'is_ct2_singaporean', 'is_ct2_spanish', 'is_ct2_ecuadorian', 'is_ct2_pizza', 'is_ct2_persian', 'is_ct2_asian', 'is_ct2_latin_american', 'is_ct2_barbecue', 'is_ct2_ice_cream_gelato', 'is_ct2_mexican', 'is_ct2_cafe', 'is_ct2_thai', 'is_ct2_caribbean', 'is_ct2_turkish', 'is_ct2_tapas_small_plates', 'is_ct2_burmese', 'is_ct2_desserts_bakeries', 'is_ct2_south_american', 'is_ct2_gastropub_food', 'is_ct2_peruvian', 'is_ct2_indian', 'is_ct2_korean', 'is_ct2_colombian', 'is_ct2_european', 'is_ct2_british', 'is_ct2_indonesian', 'is_ct2_balkan', 'is_ct2_srilankan', 'is_ct2_hawaiian', 'is_ct2_jewish', 'is_ct2_taiwanese', 'is_ct2_african', 'is_ct2_middle_eastern', 'is_ct2_vegan', 'is_ct2_asian_noodle_soup', 'is_ct2_seafood', 'is_ct2_french', 'is_ct2_polynesian', 'is_ct2_russian', 'is_ct2_brunch', 'is_ct2_australian', 'is_ct2_cuban', 'is_ct2_filipino', 'is_ct2_vegetarian', 'is_ct2_vietnamese', 'is_ct2_malaysian', 'is_ct2_lao', 'is_ct2_health_food', 'is_ct2_north_african', 'is_ct2_greek', 'is_ct2_american', 'is_ct2_east_european', 'is_ct2_nordic', 'is_ct2_north_american', 'is_ct2_italian', 'is_ct2_other', 'is_ct2_soul_food', 'is_ct2_californian', 'is_ct3_portuguese', 'is_ct3_mexican', 'is_ct3_chinese', 'is_ct3_thai', 'is_ct3_peruvian', 'is_ct3_central_american', 'is_ct3_mediterranean', 'is_ct3_japanese', 'is_ct3_singaporean', 'is_ct3_persian', 'is_ct3_asian', 'is_ct3_latin_american', 'is_ct3_spanish', 'is_ct3_ice_cream_gelato', 'is_ct3_barbecue', 'is_ct3_south_american', 'is_ct3_caribbean', 'is_ct3_vietnamese', 'is_ct3_tapas_small_plates', 'is_ct3_desserts_bakeries', 'is_ct3_gastropub_food', 'is_ct3_other', 'is_ct3_korean', 'is_ct3_health_food', 'is_ct3_european', 'is_ct3_indonesian', 'is_ct3_indian', 'is_ct3_hawaiian', 'is_ct3_jewish', 'is_ct3_taiwanese', 'is_ct3_north_american', 'is_ct3_middle_eastern', 'is_ct3_french', 'is_ct3_asian_noodle_soup', 'is_ct3_vegan', 'is_ct3_german', 'is_ct3_brunch', 'is_ct3_cuban', 'is_ct3_filipino', 'is_ct3_vegetarian', 'is_ct3_turkish', 'is_ct3_malaysian', 'is_ct3_north_african', 'is_ct3_hispanic', 'is_ct3_american', 'is_ct3_pizza', 'is_ct3_african', 'is_ct3_californian', 'is_ct3_seafood', 'is_ct3_soul_food', 'is_ct3_italian']], on='meal_id', how='inner')

### Add features from text processing

In [46]:
menus['about'].fillna('', inplace=True)

In [47]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, ngram_range=(1,1))

In [53]:
vect = vectorizer.fit_transform(menus.about)
feature_names = vectorizer.get_feature_names()
latent_features = run_sklearn_nmf(vect, feature_names, 7)

components: [[  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   1.05424806e-02
    4.25373224e-03   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  2.76391518e-04   0.00000000e+00   0.00000000e+00 ...,   1.91517899e-05
    0.00000000e+00   0.00000000e+00]
 ..., 
 [  3.24980712e-03   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  6.37102010e-04   5.85649461e-02   0.00000000e+00 ...,   7.43774258e-03
    0.00000000e+00   5.56750519e-03]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]]
(7, 15433)
[u'use', u'feature', u'come', u'experience', u'ingredient', u"'s", u'cuisine', u'flavor', u'menu', u'dish']
[u'shape', u'new', u'dry', u'dente', u'al', u'italian', u'friend', u'baia', u'pasta', u'tester']
[u'kamayan', u'local', u'food', u'school', u'philippine', u'gather', u'explore', u'salo', u'